# import

In [1]:
import sys
import os
import pandas as pd
import seaborn as sns
import numpy as np
from src.constants import *
from src.utility import *
from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
pgModel = PerGameModel()
initPG = PerGameInit()

# configure

In [2]:
season = "20_22"
situation = "5on5"

groupby_cols = ["team", "season"]
num_games = 5
season_rolling = True

target = 'next_reg_win'
target_cols = ['reg_win']
target_operations = []

null_target_value = 2

# Both = True if want to include final games of season
include_null_next = True
include_null_targets = True

In [3]:
df = pd.read_csv(CSV_DB_PATH + f"{season}/PER_GAME_BY_TEAM_{situation}_{season}.csv")

if include_null_targets:
    df = pgModel.add_target(df, target, target_cols, target_operations, include_null_targets)
    df[target].fillna(null_target_value, inplace = True)
else:
    df = pgModel.add_target(df, target, target_cols, target_operations)
    
selected_cols = df.columns[~df.columns.isin(IGNORED_COLS)]

rolling = RollingData(df.copy(), selected_cols, num_games, include_null_next, groupby_cols)
rolling.essentials.to_csv(CSV_TEMP_PATH + f'ROLLING_{num_games}_{situation}_{season}.csv', header=True, index=False)

KeyError: 0

# test

In [8]:
df_merged = merge_data(df.copy(), rolling.df.copy())

# adding next values... not needed for a rolling 
df_merged = add_next_col(df_merged.copy(), 'is_home')
df_merged = add_next_col(df_merged.copy(), 'opp_team')
df_merged = add_next_col(df_merged.copy(), 'game_date')

df_full = merge_team_data(df_merged.copy(), rolling.cols, "team", "next_opp_team", "next_game_date")

#rolling.essentials

In [1]:
rolling.essentials

NameError: name 'rolling' is not defined

# export

In [13]:
## ROLLING ESSENTIALS
rolling.essentials.to_csv(CSV_DB_PATH + f'{season}/ROLLING_{num_games}_{situation}_{season}.csv', header=True, index=False)
os.remove(CSV_TEMP_PATH + f'ROLLING_{num_games}_{situation}_{season}.csv')